In [ ]:
import featuretools as ft
from featuretools.primitives import RollingMean, NumericLag, RollingMin
from featuretools.demo.weather import load_weather

# Feature Engineering for Time Series Problems

Time series forecasting consists of predicting future values of a target using earlier observations. In datasets that are used in time series problems, there is an inherent temporal ordering to the data (determined by a time index), and  the sequential target values we're predicting are highly dependent on one another. Feature engineering for time series problems exploits the fact that more recent observations are more predictive than more distant ones.

This guide will explore how to use Featuretools for automating feature engineering for univariate time series problems, or problems in which only the time index and target column are included.
 
We'll be working with a temperature demo dataset of minimum daily temperatures that includes two columns: `Temp` and `Date`. `Date` is our time index, and `Temp` is our target column. The engineered features will be built from our target, `Temp`, and will be used to help us predict future temperatures. 

In [ ]:
es = load_weather()

es['temperatures'].head(10)

Other time-related concepts in Featuretools will not be relevant here, as this is single table, and time series problems look backwards into the past of an individual column to understand observations within that column, so the concepts of cutoff times and last time indices do not exist in the same way. A cutoff time assumes that non-instantaneous values start at their time index and extend forwards in time. References to that value in child dataframes determine how far forwards that value extends until the last time index is reached.

<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=100% src="../_static/images/multi_table_FE_timeline.png" alt="Featuretools" />
</p>

Therefore, instead of using an target value's `time_index` and `cutoff_time` to determine the earliest and latest dates for the feature engineering window, we will define a window that precedes the instance using new concepts of a `gap` and `window_length`.


## Gap and Window Length

Note that we will be using integers when definint gap an window length. This implies that our data occurs at evenly spaced intervals--in this case daily--so a number `n` corresponds to `n` days. Support for unevenly spaced intervals is ongoing.

If we are at a point in time `t`, we have access to information from times less than `t` (past instances), and we do not have information from times greater than `t` (future instances). Our limitations in feature engineering, then, will come from when exactly before `t` we have access to the data. 

Consider an example where we're recording data that takes a week to ingest; the earliest data we have access to is from seven days ago, or `t - 7`. We'll call this our `gap`. A `gap` of 0 would include the instance itself, which we must be careful to avoid in time series problems, as this exposes our target.

We also need to determine how far back in time before `t - 7` we can go. Too far back, and we may lose the potency of our recent observations, but too recent, and we may not capture the full spectrum of behaviors displayed by the data. In this example, let's say that we only want to look at 5 days worth of data at a time. We'll call this our `window_length`. 

<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=100% src="../_static/images/time_series_FE_timeline.png" alt="Featuretools" />
</p>

In [ ]:
gap = 7
window_length = 5

With these two parameters (`gap` and `window_length`) set, we have defined our feature engineering window. Now, we can move onto defining our feature primitives.

## Time Series Primitives

There are three types of primitives we'll focus on for time series problems. One of them will extract features from the time index, and the other two types will extract features from our target column. 

### Datetime Transform Primitives

We need a way of implicating time in our time series features. Yes, using recent temperatures is incredibly predictive in determining future temperatures, but there is also a whole host of historical data suggesting that the month of the year is a pretty good indicator for the temperature outside. However, if we look at the data, we'll see that, though the day changes, the observations are always taken at the same hour, so the `Hour` primitive will not likely be useful. Of course, in a dataset that is measured at an hourly frequency or one more granular, `Hour` may be incrediby predictive. 

In [ ]:
datetime_primitives = ['Day', "Year", "Weekday", "Month"]

### Delaying Primitives

The simplest thing we can do with our target column is to build features that are delayed (or lagging) versions of the target column. We'll make one feature per observation in our feature engineering windows, so we'll range over time from `t - gap - window_length` to `t - gap`. 

For this purpose, we can use our `NumericLag` primitive and create one primitive for each instance in our window. 

In [ ]:
delaying_primitives = [NumericLag(periods=i + gap) for i in range(window_length)]

### Rolling Transform Primitives

Since we have access to the entire feature engineering window, we can aggregate over that window. Featuretools has several rolling primitives with which we can achieve this. Here, we'll use the `RollingMean` primitives `RollingMin`, setting the `gap` and `window_length` accordingly. Here, the gap is incredibly important, because when the gap is zero, it means the current observation's taret value is present in the window, which exposes our target.

This concern also exists for other primitives that reference earlier values in the dataframe. Because of this, when using primitives for time series feature engineering, one must be incredibly careful to not use primitives on the target column that incorporate the current observation when calculating a feature value.

In [ ]:
rolling_mean_primitive = RollingMean(window_length=window_length, 
                                     gap=gap,
                                     min_periods=window_length)

rolling_min_primitive = RollingMin(window_length=window_length, 
                                     gap=gap,
                                     min_periods=window_length)

Below is an example of how we can extract many features using the same feature engineering window without exposing our target value.

<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=100% src="../_static/images/window_calculations.png" alt="Featuretools" />
</p>

## Run DFS

Now that we've definied our time series primitives, we can pass them into DFS and get our feature matrix! 

In [ ]:
fm, f = ft.dfs(entityset=es,
               target_dataframe_name='temperatures',
               trans_primitives = (datetime_primitives + 
                                  delaying_primitives + 
                                  [rolling_mean_primitive, rolling_min_primitive]),
               max_depth=1
              )

f

In [ ]:
fm.iloc[:,[0,2, 6, 7, 8, 9]].head(15)